In [1]:
# !pip install langchain
# !pip install chromadb
# !pip install transformers
# !pip install sentence_transformers

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader

In [4]:
loader = TextLoader('./Radar Basics1.txt')
documents = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "bigscience/bloom-560m"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# with open('drive/My Drive/Radar Basics1.txt', 'r') as file:


In [22]:
embeddings = hf
vectordb = Chroma.from_documents(texts, embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [28]:
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM

class bloom(LLM):

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
        model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")
        inputs = tokenizer(prompt, return_tensors="pt").input_ids
        outputs = model.generate(inputs, max_new_tokens=7, do_sample=True, top_k=50, top_p=0.95).tolist()[0]
        return (tokenizer.decode(outputs))

In [30]:
qa = VectorDBQA.from_chain_type(llm=bloom(), chain_type="stuff", vectorstore=vectordb)

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:245: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [31]:
query = "What is Radar?"
qa.run(query)

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nsignals  or,  more commonly,  low duty -cycle pulsed signals.\n\nELINT  experts sometimes scrutinize these characteristics, as they  provide additi onal information about a radar's characteristics .\n  www.tektronix.com/radar  5  Pulse top amplitude (Power) and Pulse Width (PW) are  important for calculating the total energy in a given pulse  (Power x Time).\nKnowing the duty cycle and the power of a  given pulse, the average RF power t ransmitted can be  calculated (Pulse Power x Duty Cycle).\n  Radar Equation   The Radar Equation , as it is commonly known, defines many  of the engineering trade -offs encountered  by radar designers.\n    Equation 1.\nRadar Range Equation   The radar equation relates the expected receive  power (P r) to  the transmitted pulse p ower (P t); based on transmit antenna  gain (G t), area of t